In [1]:
import pandas as pd
import numpy as np
books = pd.read_csv('books_cleaned.csv')
from transformers import pipeline

In [3]:
books['categories'].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Aged women,1
475,Imperialism,1
476,Human-animal relationships,1
477,Amish,1


In [5]:
books['categories'].value_counts().reset_index().query('count>50')

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [6]:
category_mapping = {
    'Fiction': 'Fiction',	
	"Juvenile Fiction": "Children's Fiction",	
	'Biography & Autobiography': 'Nonfiction',
    'History': 'Nonfiction',
	'Literary Criticism': 'Nonfiction',
	'Religion': 'Nonfiction',
	'Philosophy': 'Nonfiction',
	'Comics & Graphic Novels': 'Fiction',
	'Drama':'Fiction',
	'Juvenile Nonfiction': "Children's Nonfiction",
	'Science': 'Nonfiction',
	'Poetry': 'Fiction'	
}
books['simple_categories'] = books['categories'].map(category_mapping)

In [7]:
fiction_categories = ['fiction', 'non-fiction']
pipe = pipeline('zero-shot-classification',
               model = 'facebook/bart-large-mnli')

Device set to use cpu


In [10]:
sequence = books.loc[books['simple_categories']=='Fiction', 'description'].reset_index(drop=True)[0]

In [11]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [12]:
max_index = np.argmax(pipe(sequence, fiction_categories)['scores']) # returns index of the highest probability
max_label = pipe(sequence, fiction_categories)['labels'][max_index]
max_label

'fiction'

In [13]:
def generate_prediction(sequence, categories):
    predictions = pipe(sequence, fiction_categories)
    max_index = np.argmax(predictions['scores']) 
    max_label = predictions['labels'][max_index]
    return max_label
    

In [14]:
from tqdm import tqdm 
actual_cats = []
predicted_cats = []
for i in tqdm(range(300)):
    sequence = books.loc[books['simple_categories']=='Fiction', 'description'].reset_index(drop=True)[i]
    predicted_cats+= generate_prediction(sequence, fiction_categories)
    actual_cats+= 'Fiction'

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [26:29<00:00,  5.30s/it]


In [15]:
for i in tqdm(range(300)):
    sequence = books.loc[books['simple_categories']=='Nonfiction', 'description'].reset_index(drop=True)[i]
    predicted_cats+= generate_prediction(sequence, fiction_categories)
    actual_cats+= 'Nonfiction'

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [26:51<00:00,  5.37s/it]


In [20]:
print(len(actual_cats), len(predicted_cats))


5100 6020


In [19]:
predictions_df = pd.DataFrame({'actual_categories': actual_cats, 'predicted_categories':predicted_cats})
predictions_df

ValueError: All arrays must be of the same length

In [ ]:
predictions_df['correct_prediction'] = np.where(predictions_df['actual_categories']==predictions_df['predicted_categories'],1,0) 

In [ ]:
predictions_df['correct_prediction'].sum()/len(predictions_df)

In [ ]:
isbns = [] 
predicted_cats = []
missing_cats = books.loc[books['simple_categories'].isna(), ['isbn13', 'description']].reset_index(drop = True)